In [ ]:
import sys
sys.path.insert(0, '..')

import polars as pl
from pathlib import Path

# Load the same preprocessed data as topic modeling to ensure matching IDs
# This ensures sentiment analysis runs on the same speeches as topic modeling
processed_dir = Path('../data/processed')
df = pl.read_parquet(processed_dir / 'df_sample_split_preprocessed_topic.parquet')

print(f"df_sample_split_preprocessed_topic loaded!")
print(f"Shape: {df.shape}")
print(f"\nColumn names: {df.columns}")
print(f"\nData types:")
print(df.schema)
print(f"\nFirst few rows:")
print(df.head())

df_sample_split loaded!
Shape: (3142, 15)

Column names: ['id', 'session', 'electoralTerm', 'firstName', 'lastName', 'politicianId', 'speechContent', 'factionId', 'documentUrl', 'positionShort', 'positionLong', 'date', 'speech_length', 'paragraph_number', 'paragraph_length']

Data types:
Schema([('id', Int64), ('session', Int64), ('electoralTerm', Int64), ('firstName', String), ('lastName', String), ('politicianId', Int64), ('speechContent', String), ('factionId', Int64), ('documentUrl', String), ('positionShort', String), ('positionLong', String), ('date', String), ('speech_length', Int64), ('paragraph_number', Int64), ('paragraph_length', Int64)])

First few rows:
shape: (5, 15)
┌────────┬─────────┬────────────┬───────────┬───┬────────────┬────────────┬────────────┬───────────┐
│ id     ┆ session ┆ electoralT ┆ firstName ┆ … ┆ date       ┆ speech_len ┆ paragraph_ ┆ paragraph │
│ ---    ┆ ---     ┆ erm        ┆ ---       ┆   ┆ ---        ┆ gth        ┆ number     ┆ _length   │
│ i64  

In [ ]:
# trying the sentiment API

from germansentiment import SentimentModel

model = SentimentModel()

texts = [
    "Mit keinem guten Ergebniss","Das ist gar nicht mal so gut",
    "Total awesome!","nicht so schlecht wie erwartet",
    "Der Test verlief positiv.","Sie fährt ein grünes Auto."]
       
result = model.predict_sentiment(texts)
print(result)

classes, probabilities = model.predict_sentiment(["das ist super"], output_probabilities = True) 
print(classes, probabilities)


c:\Users\olive\miniconda3\envs\parl_speech\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['negative', 'negative', 'positive', 'positive', 'neutral', 'neutral']
['positive'] [[['positive', 0.9761366844177246], ['negative', 0.02354043908417225], ['neutral', 0.0003229434078093618]]]


In [ ]:
# Cut paragraphs longer than 300 words using native Polars (10-100x faster than map_elements)
import polars as pl
from src.utils.text_processing import trim_to_max_words_native, add_word_count

print("Trimming paragraphs longer than 300 words...")

# Apply trimming using native Polars operations (much faster than map_elements)
df_trimmed = trim_to_max_words_native(df, 'speechContent', max_words=300)

# Calculate word counts for statistics
df_trimmed = add_word_count(df_trimmed, 'speechContent', 'word_count')

print(f"\nTrimming complete!")
print(f"Total rows: {df_trimmed.shape[0]}")
print(f"\nWord count statistics after trimming:")
print(f"  Min words: {df_trimmed['word_count'].min()}")
print(f"  Max words: {df_trimmed['word_count'].max()}")
print(f"  Mean words: {df_trimmed['word_count'].mean():.1f}")
print(f"  Median words: {df_trimmed['word_count'].median():.1f}")

print(f"\nFirst few rows after trimming:")
print(df_trimmed.select(['id', 'word_count', 'speechContent']).head(5))

Trimming paragraphs longer than 300 words...

Trimming complete!
Total rows: 3142

Word count statistics after trimming:
  Min words: 1
  Max words: 300
  Mean words: 48.8
  Median words: 27.0

First few rows after trimming:
shape: (5, 3)
┌────────┬────────────┬─────────────────────────────────┐
│ id     ┆ word_count ┆ speechContent                   │
│ ---    ┆ ---        ┆ ---                             │
│ i64    ┆ u32        ┆ str                             │
╞════════╪════════════╪═════════════════════════════════╡
│ 739083 ┆ 7          ┆ Herr Präsident! Liebe Kollegin… │
│ 739083 ┆ 63         ┆ total leeren Tribünen - mit Au… │
│ 739083 ┆ 1          ┆ ({0})                           │
│ 739083 ┆ 8          ┆ Der Grund: So soll ein Skandal… │
│ 739083 ┆ 1          ┆ ({1})                           │
└────────┴────────────┴─────────────────────────────────┘


In [ ]:
# Calculate sentiment for each paragraph using batch processing (faster)
from germansentiment import SentimentModel
from src.utils.text_processing import batch_process_sentiment
import polars as pl

print("Loading sentiment model...")
model = SentimentModel()

print(f"Processing {len(df_trimmed)} paragraphs for sentiment analysis...")

# Extract texts
texts = df_trimmed['speechContent'].to_list()

# Use batch processing for better performance
sentiments, probabilities_list = batch_process_sentiment(texts, model, batch_size=32)

# Add sentiment columns to dataframe
df_sentiment = df_trimmed.with_columns([
    pl.Series('sentiment_class', sentiments),
    pl.Series('sentiment_probabilities', probabilities_list, dtype=pl.Object)
])

print(f"\nSentiment analysis complete!")
print(f"Total paragraphs: {len(df_sentiment)}")

# Show sentiment distribution
print(f"\nSentiment class distribution:")
print(df_sentiment['sentiment_class'].value_counts().sort('count', descending=True))

# Extract individual probabilities for better visualization
print(f"\nFirst few results:")
print(df_sentiment.select(['id', 'word_count', 'sentiment_class', 'sentiment_probabilities']).head(5))

Loading sentiment model...
Processing 3142 paragraphs for sentiment analysis...
  Processed 3142 texts in 99 batches

Sentiment analysis complete!
Total paragraphs: 3142

Sentiment class distribution:
shape: (3, 2)
┌─────────────────┬───────┐
│ sentiment_class ┆ count │
│ ---             ┆ ---   │
│ str             ┆ u32   │
╞═════════════════╪═══════╡
│ neutral         ┆ 2125  │
│ positive        ┆ 705   │
│ negative        ┆ 312   │
└─────────────────┴───────┘

First few results:
shape: (5, 4)
┌────────┬────────────┬─────────────────┬─────────────────────────────────┐
│ id     ┆ word_count ┆ sentiment_class ┆ sentiment_probabilities         │
│ ---    ┆ ---        ┆ ---             ┆ ---                             │
│ i64    ┆ u32        ┆ str             ┆ object                          │
╞════════╪════════════╪═════════════════╪═════════════════════════════════╡
│ 739083 ┆ 7          ┆ neutral         ┆ [['positive', 0.00488126790151… │
│ 739083 ┆ 63         ┆ neutral         ┆ [

In [ ]:
# Save sentiment analysis results (parquet only - more efficient)
import polars as pl
from pathlib import Path
import json

# Define output path
output_dir = Path('../data/processed')
output_dir.mkdir(parents=True, exist_ok=True)

# Convert probabilities to JSON strings using map_elements
df_to_save = df_sentiment.with_columns(
    pl.col('sentiment_probabilities').map_elements(
        lambda x: json.dumps(x) if x is not None else None,
        return_dtype=pl.Utf8
    ).alias('sentiment_probabilities')
)

# Save as Parquet only (more efficient than CSV)
output_file = output_dir / 'df_sample_sentiment.parquet'
df_to_save.write_parquet(output_file)

print(f"✓ Sentiment analysis results saved!")
print(f"  File: {output_file}")
print(f"  Rows: {len(df_to_save)}")
print(f"  Columns: {len(df_to_save.columns)}")

# Display summary statistics
print(f"\n📊 Sentiment Analysis Summary:")
print(f"  Total paragraphs analyzed: {len(df_sentiment)}")
print(f"\n  Sentiment Distribution:")
sentiment_counts = df_sentiment['sentiment_class'].value_counts().sort('count', descending=True)
for row in sentiment_counts.iter_rows(named=True):
    percentage = (row['count'] / len(df_sentiment)) * 100
    print(f"    {row['sentiment_class']:10s}: {row['count']:3d} ({percentage:5.1f}%)")

print(f"\n✓ Data ready for further analysis!")

✓ Sentiment analysis results saved!
  File: ..\data\processed\df_sample_sentiment.parquet
  Rows: 3142
  Columns: 18

📊 Sentiment Analysis Summary:
  Total paragraphs analyzed: 3142

  Sentiment Distribution:
    neutral   : 2125 ( 67.6%)
    positive  : 705 ( 22.4%)
    negative  : 312 (  9.9%)

✓ Data ready for further analysis!
